In [1]:
import sys
sys.path.append("../")

import logging
import pickle
import numpy as np
import pandas as pd

from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import rmse
from reco_utils.recommender.sar import SAR

In [2]:
# !wget http://www.occamslab.com/petricek/data/ratings.dat

In [3]:
# top k items to recommend
TOP_K = 10
data = pd.read_csv('../Data/ratings_compressed.csv', names=['user_id', 'profile_id', 'rating'])
data['rating'] = data['rating'].astype(np.float32)

In [4]:
len(data)

110023

In [5]:
train, test = python_stratified_split(data, ratio=0.75, col_user='user_id', col_item='profile_id', seed=42)

In [6]:
# train.to_csv('../Data/train.csv', header=False, index=False)
# test.to_csv('../Data/test.csv', header=False, index=False)

In [7]:
model = SAR(
    col_user="user_id",
    col_item="profile_id",
    col_rating="rating",
    similarity_type="jaccard",
    time_decay_coefficient=0,
    normalize=True
)

In [8]:
model.fit(train)

In [9]:
top_k = model.recommend_k_items(test, remove_seen=True, normalize=True) # if we want just top K for users
score = model.score(test, normalize=True) # if we want to predict all items for users
predictions = model.predict(test, normalize=True) # if we want to predict the test set only, MAYBE NOT NORMALIZED TO SCALE ORIGINAL RATINGS!

../reco_utils/recommender/sar/sar_singlenode.py:333: RuntimeWarning: invalid value encountered in true_divide
  self.unity_user_affinity[user_ids, :].dot(self.item_similarity),
Items found in test not seen during training, new items will have score of 0


In [10]:
predictions = predictions.replace([np.inf, -np.inf], 0)

In [11]:
rmse(test, predictions, col_user='user_id', col_item='profile_id', col_rating='rating', col_prediction='prediction')

2.861099430521089

In [12]:

f = open('model/SAR.pickle', 'wb')
pickle.dump(model, f)  
f.close()

In [13]:
f = open('model/SAR.pickle','rb') 
loaded_model = pickle.load(f)
f.close()

In [14]:
predictions = loaded_model.predict(test, normalize=True)

../reco_utils/recommender/sar/sar_singlenode.py:333: RuntimeWarning: invalid value encountered in true_divide
  self.unity_user_affinity[user_ids, :].dot(self.item_similarity),
Items found in test not seen during training, new items will have score of 0


In [15]:
predictions = predictions.replace([np.inf, -np.inf], 0)

In [16]:
rmse(test, predictions, col_user='user_id', col_item='profile_id', col_rating='rating', col_prediction='prediction')

2.861099430521089

In [19]:
train.to_csv('../Data/train.csv', index=False, header=False)

In [20]:
test.to_csv('../Data/test.csv', index=False, header=False)